In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.activations import relu, tanh, softmax, sigmoid
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout, PReLU, LeakyReLU, Input, BatchNormalization, Activation, Flatten


In [2]:
with open('../colume_pickle/test_complete_df_6개월_choice_10개_대형주.pkl', 'rb') as f:
    stock_df = pickle.load(f)
with open('../colume_pickle/6개월_choice_10개_cor_0.075_lstm_대형주.pkl', 'rb') as f:
    col_list = pickle.load(f)

In [3]:
modelpath="./model/lstm_100_128_10_0.0000001_0.075_대형주_범위(0,1).hdf5"

In [4]:
from keras.models import load_model
model = load_model(modelpath)

In [5]:
# pred_df['pct_label'] = pred_df['pct_label'].apply(np.floor)
# pred_df[pred_df['pct_label'] > 0] = 1
# y_pred_df = ohe.transform(pred_df[['pct_label']])
# y_pred_df.shape

In [6]:
def predict_stock(pred_df, col_list):
    col_list = list(col_list.index)
    X_pred_df = pred_df.drop(['pct_label'],axis=1)
    X_pred_df = X_pred_df.set_index(['날짜'])
    X_pred_df = X_pred_df[col_list]
    
    min_max_scaler = MaxAbsScaler()
    X_pred_sc = min_max_scaler.fit_transform(X_pred_df)
    X_pred = X_pred_sc.reshape(X_pred_sc.shape[0], 37, 1)

    predict = model.predict(X_pred)
    
    return predict

In [7]:
def stock_dataframe(code):
    with open(f'../colume_pickle/6개월_choice_10개_대형주_update_{code}.pkl', 'rb') as f:
        pred_df = pickle.load(f)
    return pred_df

In [9]:
cost = 10000000 # 잔고
stock_list = ['005930','373220','207940','000660','051910','247540','091990','066970','293490','028300']
for code in stock_list:
    pred_stock_df = stock_dataframe(code) # 종목 데이터 프레임
    predict_ = predict_stock(pred_stock_df, col_list) # 종목 예측
    
    for x in tqdm(range(len(predict_))):
        
        end_cost = pred_stock_df.loc[x,'종가']   # 종가
        high_cost = pred_stock_df.loc[x,'고가']   # 고가
        
        stock_cost = 1000000    # 종목별 매수 금액
        num = 0 # 가지고 있는 주
        # num = 매수금액 / 종가
        print(predict_[x][0] < predict_[x][1])
        if predict_[x][0] < predict_[x][1] and end_cost < high_cost and stock_cost > end_cost*buy_num:
            buy_num = stock_cost / end_cost
            stock_cost = stock_cost - (end_cost*buy_num) # 매수
            num += buy_num
        elif predict_[x][0] > predict_[x][1] :
            stock_cost = stock_cost + (high_cost * num) # 매도
            num -= num
        # else:
        #     hold

    cost = cost + (stock_cost - 1000000)
#  잔고가 얼마냐? + 수익  - 손실

12/12 [==============================] - 0s 7ms/step


100%|██████████| 381/381 [00:00<00:00, 39357.43it/s]

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals


100%|██████████| 381/381 [00:00<00:00, 46141.82it/s]

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

...

100%|██████████| 381/381 [00:00<00:00, 53724.32it/s]


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

  0%|          | 0/381 [00:00<?, ?it/s]

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

100%|██████████| 381/381 [00:00<00:00, 37822.30it/s]


In [10]:
cost

10000000.0

In [31]:
len(predict)#[0.6887641 , 0.31123596]

381

In [ ]:
pred = tf.argmax(predict,1).numpy()
test = tf.argmax(y_pred_df,1).numpy()

In [ ]:
np.unique(pred,return_counts=True)

In [ ]:
np.unique(test,return_counts=True)

In [ ]:
acc = accuracy_score(test, pred)
print('accuracy_score : ',acc)

In [ ]:

plt.plot(test, label='Training Loss')
plt.plot(pred, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,2.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()